# IMDB Project 3 Part 2 (CORE)

**Marco Jimenez**

In [34]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [35]:
!pip install tmdbsimple

In [36]:
with open('/Users/LP-Ca/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [37]:
# importing package and api-key
tmdb.API_KEY = login['api-key']

In [38]:
# Making movie object
movie = tmdb.Movies(603)
movie

In [39]:
# Looking at movie object dict
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/ncEsesgOJDNrTUED89hYbA117wo.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 64.023,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [40]:
def get_movie_with_rating(movie_id):
    # Getting movie object for current id
    movie = tmdb.Movies(movie_id)
    # Saving .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Looping through the countries in releases
    for c in releases['countries']:
        # If the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            # Saving a certification key in the info dictionaries with the certification
            info['certification'] = c['certification']
    return info

In [41]:
# Testing function
get_movie_with_rating(501)

{'adult': False,
 'backdrop_path': '/hAlB9MaamrP1eDhGA4AkjSAPxAf.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 99, 'name': 'Documentary'}],
 'homepage': '',
 'id': 501,
 'imdb_id': 'tt0427312',
 'original_language': 'en',
 'original_title': 'Grizzly Man',
 'overview': 'Werner Herzog\'s documentary film about the "Grizzly Man" Timothy Treadwell and what the thirteen summers in a National Park in Alaska were like in one man\'s attempt to protect the grizzly bears. The film is full of unique images and a look into the spirit of a man who sacrificed himself for nature.',
 'popularity': 10.895,
 'poster_path': '/nXIV2qGK9KkdkaOTzrpK87CuAGC.jpg',
 'production_companies': [{'id': 274,
   'logo_path': None,
   'name': 'Discovery Docs',
   'origin_country': ''}],
 'production_countries': [{'iso_3166_1': 'US',
   'name': 'United States of America'}],
 'release_date': '2005-08-12',
 'revenue': 0,
 'runtime': 103,
 'spoken_languages': [{'english_name': 'English',
   'iso_6

In [42]:
# Designating folder to save info
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [43]:
# Reading in filtered basics csv
basics = pd.read_csv('/Users/LP-Ca/Documents/DataEnrichment/IMDB-Movie-Database-Project/Data/title_basics.csv.gz')
basics.head()  

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [44]:
# Defining range of years to retrieve
yrs_to_get = [2000,2001]

In [46]:
YEAR = yrs_to_get[0]
YEAR

2000

In [47]:
# Beginning outer loop
for YEAR in tqdm_notebook(yrs_to_get, desc='YEARS',position=0):

SyntaxError: unexpected EOF while parsing (1616811233.py, line 2)

In [48]:
# Dsignating JSON file to store results
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Checking if file exists
file_exists = os.path.isfile(JSON_FILE)

In [50]:
# Creating file in case it doesn't exist
if file_exists == False:
    # Saving empty dict w/ imdb_id to new json file
    with open(JSON_FILE, 'w') as f:
         json.dump([{'imdb_id':0}],f)

In [51]:
# Saving new year as the current df
df = basics.loc[basics['startYear']==YEAR].copy()
# Saving movie id's to list
movie_ids = df['tconst'].copy()

In [52]:
# Loading existing data from json into a dataframe
prev_df = pd.read_json(JSON_FILE)
prev_df

,imdb_id
0,0


In [53]:
# Filtering out id's that are already in JSON_FILE
get_movieid = movie_ids[~movie_ids.isin(prev_df['imdb_id'])]

In [54]:
# Defining function to be used in the loop
def write_json(new_data, filename):
    with open(filename, 'r+') as file:
        # Loading existing data into a dict
        file_data = json.load(file)
        # Append or extend
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Setting files current position at offset
        file.seek(0)
        # Converting back to .json
        json.dump(file_data, file)

In [ ]:
#Getting index and movie id from list
for movie_id in tqdm_notebook(get_movieid,
                              desc=f'Movies from {YEAR}',
                              position=1,
                              leave=True):
    # Attempting to retrieve data for movie_id
    try:
        # Using premade movie_id function
        temp = get_movie_with_rating(movie_id) 
        # Applying premade write_json function to appen/extend
        write_json(temp,JSON_FILE)
        # Applying a short 20ms sleep to prevent overwhelming server
        time.sleep(0.02)

    # If it fails, make a dict with just the id and None for certification.
    except Exception as e:
        continue

Movies from 2000:   0%|          | 0/1405 [00:00<?, ?it/s]